In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd
from urllib.request import urlopen
import time
from selenium.webdriver.common.by import By
import requests
from selenium.webdriver.common.keys import Keys
from datetime import datetime
import xlwt

driver = webdriver.Chrome(ChromeDriverManager().install())

driver.get("https://ebook.kyobobook.co.kr/dig/etc/landing/best")

[WDM] - Downloading: 100%|██████████| 6.79M/6.79M [00:00<00:00, 9.02MB/s]
C:\Users\김근영\AppData\Local\Temp\ipykernel_15180\1083539102.py:17: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [2]:
book_list = []

genre_dict = { 
        18 : "경제경영",
        19 : "자기계발",
        20 : "시/에세이",
        21 : "인문",
        22 : "종교",
        23 : "소설",
        24 : "국어/외국어",
        25 : "정치/사회",
        26 : "역사/문화",
        27 : "과학/공학",
        28 : "IT/프로그래밍",
        29 : "건강/의학",
        30 : "가정/생활/요리",
        31 : "여행/취미",
        32 : "예술/대중문화",
        33 : "유아",
        34 : "아동",
        35 : "청소년",
        36 : "교재/수험서",
        37 : "외국도서",
        38 : "매거진"
}

#100개씩 보기 선택
driver.find_element(By.ID,"ebookLanding_pageCnt-button").click()
time.sleep(0.5)
    
driver.find_element(By.ID,"ui-id-58").click()
time.sleep(0.5)

# 장르 선택
for genre_num in range(18,39):
    driver.find_element(By.ID,"ebookLanding_msc-button").click()
    time.sleep(0.5)
    
    driver.find_element(By.ID,f"ui-id-{genre_num}").click()
    time.sleep(0.5)
    
    bsObject = BeautifulSoup(driver.page_source, 'html.parser')
    
    books = bsObject.select('div.prodDt')
    
    for book in books:
        title = book.select_one('strong > h3 > a').text  #책 제목
        
        author = book.select_one('p.prodDt_info > span').text  #저자
        
        publisher = book.select('p.prodDt_info > span')[1].text #출판사
        
        date = book.select('p.prodDt_info > span')[2].text  #등록일
        
        price=book.select_one('div.prodDt_price span b').text+"원"# 가격
        
        if(book.select_one('div.prodDt_review > p > b') == None) : #만약 별점이 없으면 0으로
            review = 0
            
        else :
            review = book.select_one('div.prodDt_review > p > b').text #별점
            
        genre=genre_dict[genre_num]
        
        book_list.append([title,author,publisher,date,price,review,genre])

ElementClickInterceptedException: Message: element click intercepted: Element <div id="ui-id-21" tabindex="-1" role="option" class="ui-menu-item-wrapper">...</div> is not clickable at point (646, 368). Other element would receive the click: <div class="eBook_list_body" id="prdList">...</div>
  (Session info: chrome=109.0.5414.121)
Stacktrace:
Backtrace:
	(No symbol) [0x00FE6643]
	(No symbol) [0x00F7BE21]
	(No symbol) [0x00E7DA9D]
	(No symbol) [0x00EB92B2]
	(No symbol) [0x00EB6970]
	(No symbol) [0x00EB3E0B]
	(No symbol) [0x00EB2548]
	(No symbol) [0x00EA6B13]
	(No symbol) [0x00ECFD7C]
	(No symbol) [0x00EA641F]
	(No symbol) [0x00ED00D4]
	(No symbol) [0x00EE6B09]
	(No symbol) [0x00ECFB76]
	(No symbol) [0x00EA49C1]
	(No symbol) [0x00EA5E5D]
	GetHandleVerifier [0x0125A142+2497106]
	GetHandleVerifier [0x012885D3+2686691]
	GetHandleVerifier [0x0128BB9C+2700460]
	GetHandleVerifier [0x01093B10+635936]
	(No symbol) [0x00F84A1F]
	(No symbol) [0x00F8A418]
	(No symbol) [0x00F8A505]
	(No symbol) [0x0

In [ ]:
df.to_csv("kyobo_genre_best_0215.csv", index = False)

In [ ]:
import requests
from bs4 import BeautifulSoup
import csv

# 알라딘 주간 베스트 100권 크롤링
# 2/2 ~ 2/8 1주 기준

# 책 정보 추출 함수
def extract_book(html):  
    subject=html.select_one("div.ss_book_list a.bo3").text
    
    info=html.select("div.ss_book_list li")[0].text
    
    if info=="ePub"or info=="PDF" or info=="무료PDF" or info=="대여ePub":
        
        info=html.select("div.ss_book_list li")[2].text
        
    else:
        
        info=html.select("div.ss_book_list li")[3].text
        
    writer=info.split('|')[0]
    
    publisher=writer.split('|')[1]
    
    date=publisher.split('|')[2]
    
    stars=html.select_one("div.ss_book_list img")
    
    star=0
    
    if stars==None:
        
        star=0
        
    elif stars['src']=="//image.aladin.co.kr/img/common/star_s10.gif":
        
        star=5
        
    elif stars['src']=="//image.aladin.co.kr/img/common/star_s9.gif":
        
        star=4.5
        
    elif stars['src']=="//image.aladin.co.kr/img/common/star_s8.gif":
        
        star=4
        
    elif stars['src']=="//image.aladin.co.kr/img/common/star_s7.gif":
        
        star=3.5
        
    elif stars['src']==f"//image.aladin.co.kr/img/common/star_s6.gif":
        
        star=3
        
    elif stars['src']==f"//image.aladin.co.kr/img/common/star_s5.gif":
        
        star=2.5

    elif stars['src']==f"//image.aladin.co.kr/img/common/star_s4.gif":
        
        star=2
    
    return {
        'subject': subject,
        'writer':writer,
        'publisher':publisher,
        'date':date,
        'star':star
    }

# 각 페이지마다 책 정보 추출 함수
def extract_books(page_count):
    books = []
    
    for page in range(page_count):
    
        print(f"Scrapping page {page+1}")
        
        rs = requests.get(f"https://www.aladin.co.kr/shop/common/wbest.aspx?BestType=EBookBestseller&BranchType=9&CID=0&page={page+1}&cnt=300&SortOrder=1")
        
        soup = BeautifulSoup(rs.text, "html.parser")
        
        results=soup.find_all("div",class_="ss_book_box")
        
        for rs in results:
            
            book = extract_book(rs)
            
            books.append(book)
            
    return books

# CSV 저장
# 1-50 / 51-100 두 페이지만 진행
books=extract_books(2)

file=open("aladin_best100.csv",mode="w")

write=csv.writer(file)

write.writerow(["subject", "writer", "publisher","date", "star"])

for book in books:
    
    write.writerow(list(book.values()))

In [ ]:
import requests
from bs4 import BeautifulSoup
import csv

# 알라딘 주간 베스트 100권 크롤링
# 2/2 ~ 2/8 1주 기준

# 책 정보 추출 함수
def extract_book(html):  
    subject=html.select_one("div.ss_book_list a.bo3").text
    
    info=html.select("div.ss_book_list li")[0].text
    
    if info=="ePub"or info=="PDF" or info=="무료PDF" or info=="대여ePub":
        
        info=html.select("div.ss_book_list li")[2].text
        
    else:
        
        info=html.select("div.ss_book_list li")[3].text
        
    writer=info.split('|')[0]
    
    publisher=writer.split('|')[1]
    
    date=publisher.split('|')[2]
    
    stars=html.select_one("div.ss_book_list img")
    
    star=0
    
    if stars==None:
        
        star=0
        
    elif stars['src']=="//image.aladin.co.kr/img/common/star_s10.gif":
        
        star=5
        
    elif stars['src']=="//image.aladin.co.kr/img/common/star_s9.gif":
        
        star=4.5
        
    elif stars['src']=="//image.aladin.co.kr/img/common/star_s8.gif":
        
        star=4
        
    elif stars['src']=="//image.aladin.co.kr/img/common/star_s7.gif":
        
        star=3.5
        
    elif stars['src']==f"//image.aladin.co.kr/img/common/star_s6.gif":
        
        star=3
        
    elif stars['src']==f"//image.aladin.co.kr/img/common/star_s5.gif":
        
        star=2.5

    elif stars['src']==f"//image.aladin.co.kr/img/common/star_s4.gif":
        
        star=2
    
    return {
        'subject': subject,
        'writer':writer,
        'publisher':publisher,
        'date':date,
        'star':star
    }

# 각 페이지마다 책 정보 추출 함수
def extract_books(page_count):
    books = []
    
    for page in range(page_count):
    
        print(f"Scrapping page {page+1}")
        
        rs = requests.get(f"https://www.aladin.co.kr/shop/common/wbest.aspx?BestType=EBookBestseller&BranchType=9&CID=0&page={page+1}&cnt=300&SortOrder=1")
        
        soup = BeautifulSoup(rs.text, "html.parser")
        
        results=soup.find_all("div",class_="ss_book_box")
        
        for rs in results:
            
            book = extract_book(rs)
            
            books.append(book)
            
    return books

# CSV 저장
# 1-50 / 51-100 두 페이지만 진행
books=extract_books(2)

file=open("aladin_best100.csv",mode="w")

write=csv.writer(file)

write.writerow(["subject", "writer", "publisher","date", "star"])

for book in books:
    
    write.writerow(list(book.values()))